# Spin

`ebcc` offers solvers for the restricted, unrestricted, and generalised versions of coupled cluster. Most ansatzes are covered for all three cases, with some exceptions. Rather than ambiguously packing spin components into a `tuple` or `list`, `ebcc` has a `Namespace` class which gives them an explicit label.

In [1]:
import numpy as np
from pyscf import gto, scf

mol = gto.M(atom="O 0 0 0; O 0 0 1", basis="cc-pvdz", spin=2, verbose=0)
mf = scf.UHF(mol).run()

In [2]:
import sys
from logging import StreamHandler
from ebcc.core.logging import Logger

log = Logger("main")
log.setLevel(0)
log.addHandler(StreamHandler(sys.stdout))

In [3]:
from ebcc import UEBCC

qcisd = UEBCC(mf, ansatz="QCISD", log=log)
qcisd.kernel()

print("T1 keys:", qcisd.t1.keys())
print("T2 keys:", qcisd.t2.keys())

        _
       | |
   ___ | |__    ___   ___
  / _ \| '_ \  / __| / __|
 |  __/| |_) || (__ | (__
  \___||_.__/  \___| \___|
                     1.5.0
numpy:
 > Version:  1.26.1
 > Git hash: N/A
pyscf:
 > Version:  2.6.2
 > Git hash: N/A
ebcc:
 > Version:  1.5.0
 > Git hash: N/A
OMP_NUM_THREADS = 1


UQCISD
******

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: QCISD

Space: ((9o, 19v), (7o, 21v))

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.3070521941    -149.8580936225
   1    -0.3055036550    -149.8565450833     1.549e-03     1.256e-02
   2    -0.3233793745    -149.8744208029     1.788e-02     3.772e-03
   3    -0.3168286951    -149.8678701235     6.551e-03     1.942e-03
   4    -0.3186699020    -149.8697113303     1.841e-03     1.253e-03
   5    -0.3185526911    -149.8695941195     1.172e-04     1.944e-04
   6    -0.3185465027   

Classes with higher spin symmetry can be converted to those of lower symmetry with built-in conversion routines. In cases where the broken symmetry permits a different ground state, the `kernel` can be called again to proceed with the calculation.

In [4]:
from ebcc import GEBCC

gqcisd = GEBCC.from_uebcc(qcisd)
gqcisd.kernel()


GQCISD
******

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: QCISD

Space: (16o, 40v)

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.3185511176    -149.8695925459
   1    -0.3185511188    -149.8695925472     1.226e-09     9.540e-10

Converged.

E(corr) = -0.3185511188
E(tot)  = -149.8695925472

Time elapsed: 300 ms



-0.31855111878172215

As seen in the previous tutorial, we can define spaces to control frozen and active orbitals. In unrestricted calculations, the spaces can be provided as a tuple to differentiate between the alpha and beta spin channels.

In [8]:
from ebcc import Space

occupied = mf.mo_occ[0] > 0
frozen = np.zeros_like(occupied)
active = np.zeros_like(occupied)
frozen[:2] = True
space_a = Space(occupied, frozen, active)

occupied = mf.mo_occ[1] > 0
frozen = np.zeros_like(occupied)
active = np.zeros_like(occupied)
space_b = Space(occupied, frozen, active)

space = (space_a, space_b)
print(space)

uccsd = UEBCC(mf, space=space, log=log)
uccsd.kernel()
uccsd.solve_lambda()

((9o, 19v) [(2o, 0v) frozen], (7o, 21v))

UCCSD
*****

Options:
 > e_tol:  1e-08
 > t_tol:  1e-08
 > max_iter:  200
 > diis_space:  9
 > diis_min_space:  1
 > damping:  0.0

Ansatz: CCSD

Space: ((9o, 19v) [(2o, 0v) frozen], (7o, 21v))

Solving for excitation amplitudes.

Iter   Energy (corr.)      Energy (tot.)     Δ(Energy)      Δ(Ampl.)
   0    -0.3046180187    -149.8556594471
   1    -0.3034470358    -149.8544884642     1.171e-03     1.254e-02
   2    -0.3207159600    -149.8717573884     1.727e-02     3.511e-03
   3    -0.3143837333    -149.8654251616     6.332e-03     1.876e-03
   4    -0.3161360263    -149.8671774547     1.752e-03     1.210e-03
   5    -0.3160189634    -149.8670603918     1.171e-04     1.906e-04
   6    -0.3160126178    -149.8670540461     6.346e-06     1.028e-04
   7    -0.3160169280    -149.8670583564     4.310e-06     1.689e-05
   8    -0.3160153839    -149.8670568123     1.544e-06     8.321e-06
   9    -0.3160174294    -149.8670588578     2.046e-06     2.553e